# HW2, CHEE 6397 (Data-Driven Materials Modeling)

#### **Topics**: Committee Models & Cross-Validation

#### **Due**: October 17, 2023


## Instructions

- The file you submit should be named as `hw2-<FirstName>_<LastName>-<UHID>.ipynb`, e.g. `hw2-Mingjian_Wen-00001111.ipynb`.
- Input your answer to each question in the `Answer` cell. Feel free add to more cells as needed.

### Markdown and Math

- You can use `markdown` cell to type the text part of your answers to a question. And math equations can be typed using LaTex.
- See https://gtribello.github.io/mathNET/assets/notebook-writing.html for a quick intro to markdown in Jupyter and how to write math equations.
- See https://ia.wikipedia.org/wiki/Wikipedia:LaTeX_symbols for a list of LaTex symbols.
- If you are more used to MS Word equation typesetting, you can find a graphical LaTex equations generator at: https://latexeditor.lagrida.com

### Code

- Your Python code to a question can be written using the `code` cell.
- Make sure all used packages are imported properly. For example, before submission, do `Kernel->Restart and Run All Cells...` from the menu bar to double check. If we cannot run your notebook, we won't be able to grade it.


## Scores

Problem 1

- a [10 points]:
- b [10 points]:

Problem 2

- a [10 points]:
- b [10 points]:
- c [10 points]:
- d [10 points]:

Problem 3

- a [10 points]:
- b [10 points]:
- c [10 points]:
- d [10 points]:

Total: 


---
---


## Problem 1


### (a)

`Gradient boosting` is a type of additive models that uses an ensemble of weak learners to make predictions. Instead of solving the complex minimization problem directly, it solves a sequence of simpler problems. The weak learners are added to the ensemble in a forward stage-wise manner. At each stage, a weak learner is added to the ensemble to minimize the loss function of the whole ensemble.
- (1) Briefly explain what is a weak learner.
- (2) What are the similarities and differences between `gradient boosting` and `AdaBoost`?
- (3) For regression using `gradient boosting`, if a loss function based on the difference between the target and the prediction is used (e.g. the squared error $L = [t - f(x)]^2$, where $t$ is the target and $f(x)$ is model prediction), what are the best solutions for the weak learners? Justify your answer.

### Answer

* (1) Weak learner is a simple model or algorithm that performs slightly better than random guessing ($error~probability < 1/2$).

* (2)
    * Similarities:
        * Utilize and combine multiple weak learners to produce a strong learner (committee-based models)
        * Train weak learners sequentially, in which new learners are trained to compensate the errors made by the previous ones.
        * Use assigned weights on feature data, especially with features that were incorrectly predicted (high error rate).
    
    * Differences:
        * Tree Depth: `AdaBoost` often uses shallow trees (stumps) while `Gradient Boosting` can have much deeper decision trees
        * Weight assignment:
            * `AdaBoost` updates weights to each training instance in each iteration, with the weight of misclassified instances increased and that of correctly classified ones decreased.
            * `Gradient Boosting` updates the weights each iteration in order to reduce the value of the loss function.
        * Error: `AdaBoost`'s final model are built sequentially, with new learners made by taking the previous learners' errors into account. `Gradient Boosting`'s final model is the additive combination of all the weak learners.


* (3) The loss function is $L = (t - f(x))^2$. The best solutions to minimize the loss function for the weak learners are:
$$ (\beta _m, \gamma _m) = argmin_{\beta,\gamma} \sum _{i=1} ^N L(y_i,f_{m-1}(x_i) + \beta b(x_i,\gamma)) $$
$$ (\beta _m, \gamma _m) = argmin_{\gamma} \sum _{i=1} ^N (y_i - f_{m-1}(x_i) - \beta b(x_i,\gamma))^2 $$
$$ \rightarrow \frac {\delta}{\delta \gamma}\sum _{i=1} ^N (y_i - f_{m-1}(x_i) - \beta b(x_i,\gamma))^2 = 0$$
$$ -2 \sum _{i=1} ^N (y_i - f_{m-1}(x_i) - \beta b(x_i,\gamma)) [\frac {\delta}{\delta \gamma} (\beta b(x_i,\gamma))] = 0$$

$$\rightarrow \beta b(x_i,\gamma) = \frac{1}{n} \sum _{i=1} ^N (y_i - f_{m-1}(x_i))$$
$$or~~~ \beta b(x_i,\gamma) = 0$$

### (b)

`Random forest` is a type of `bagging` algorithm that uses an ensemble of decision trees to make predictions, and each decision tree is fit to a separate bootstrapped dataset. 
- (1) What are the major differences between `random forest` and `bagging with vanilla decision trees`? Vanilla decision tree is the algorithm discussed in `Lecture 5`.
- (2) What are the advantages of using `random forest` over `bagging with vanilla decision trees`?

### Answer

* (1) `Random forest` and `bagging trees` all take advantage of bootstrap dataset. However, `random forest` selects a random subset of features for each tree, while `bagging tree` use all features for each tree.

* (2) `Random forest` uses randomized bootstrapping and feature selection, making each tree independent from one another. As a result, it has lower bias and reduce overfitting (a typical problem with decision trees). This leads to improved generalization and better accuracy than `bagging`.

---


## Problem 2

In this problem, we will use the `experimental formation enthalpy` dataset of inorganic materials to explore the `random forest` and `gradient boosting` algorithms, as well as the `cross-validation` technique.

The dataset is provided in the file `experimental_formation_enthalpy.csv`. Here is some explanation: 
- any column starting with `Magpie` is a feature
- the column `formation_enthalpy` is the target
- the column `formula` gives the chemical formula of the material, from which the features are obtained using the `matminer` package composition descriptor discussed in Lab 4. 
- the column `mpid` gives the [Materials Project](https://materialsproject.org/) ID of the material. You can know more about the material by searching the ID on the Materials Project website.

### (a)

- Load the dataset into a `pandas` dataframe.
- There are `NaN` (Not a Number) values in the target `formation_enthalpy` column. Clean the dataset by removing the rows with `NaN` values in the target column. Note, other columns like `mpid` also have `NaN` values, but you should not remove the rows with `NaN` values in those columns.
- Convert all the features to a 2D numpy array `X` and the target to a 1D numpy array `y`.

Hints: refer to `Lab 3` and `HW 1` on how to do this.
 
Then answer the following questions:
1. How many samples (data points) are there in the dataset?
2. How many features are there in the dataset?


### Answer


In [42]:
import pandas as pd
import numpy as np

In [43]:
# Define macro
TARGET = 'formation_enthalpy'
FEATURE_PREFIX = 'Magpie'

In [44]:
# Load dataset into a pandas dataframe
df = pd.read_csv("experimental_formation_enthalpy.csv")

In [45]:
# remove the rows with NaN values for formation_enthalpy
df = df.dropna(axis=0,subset=TARGET)

In [46]:
# convert target and features set to numpy arrays
target_set = df[TARGET].to_numpy()
features_set = df.loc[:,df.columns.str.contains(FEATURE_PREFIX)].to_numpy()
features_names = df.columns[df.columns.str.contains(FEATURE_PREFIX)].to_numpy()

In [47]:
# NUMBER OF SAMPLES IN THE DATASET
print('Number of valid samples in the dataset:', features_set.shape[0])
# NUMBER OF FEATURES IN THE DATASET
print('Number of features in the dataset:', features_set.shape[1])

Number of valid samples in the dataset: 1196
Number of features in the dataset: 132


### (b)

Briefly explain what is `cross-validation` and why it is useful.

### Answer  

Cross validation divides the available data into multiple folds and use of one fold as a validation set while the rest are used for training. The process is repeated several times, each with different a validation set. The average performance of all validation iterations is the estimate of the model performance.

Cross validation is useful because it prevents overfitting by training evaluating the model based on different training and validation sets. This is particularly beneficial on small dataset.

### (c)

Fit a random forest model to the `training set` using 5-fold cross validation, and evaluate the training and test `root mean squared errors`. 

Hints:

- Use `RandomForestRegressor` in the `sklearn.ensemble` module as the estimator.
- Use `cross_validate` in the `sklearn.model_selection` module to perform cross validation.
- You can use `neg_root_mean_squared_error` as the `scoring` function.



### Answer


In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

In [49]:
rf_model = RandomForestRegressor()
cv_results = cross_validate(rf_model, features_set, target_set, cv = 5,
                            scoring='neg_root_mean_squared_error',
                            return_train_score=True)

### (d)


Then answer the below questions:
1. What are the mean and standard deviation of the `training errors` from the 5-fold cross validation? 
2. What are the mean and standard deviation of the `test errors` from the 5-fold cross validation? 
3. Is the model sensitive to data splitting? Justify your answer.
4. Do you think random forest model is a good model for this dataset? Why or why not?

### Answer

In [50]:
# 1 
print('Training errors:\n\tMean = {}\n\tStandard deviation = {}'.format(cv_results['train_score'].mean(),cv_results['train_score'].std()))
# 2
print('Test errors:\n\tMean = {}\n\tStandard deviation = {}'.format(cv_results['test_score'].mean(),cv_results['test_score'].std()))

Training errors:
	Mean = -0.03621601728782425
	Standard deviation = 0.0009847859975011459
Test errors:
	Mean = -0.1719672837585516
	Standard deviation = 0.017310920889979584


* **(3)** The model is not sensitive to data splitting. Since the standard deviation of the training and test scores is about 5% and 11% of the mean, respectively, the differences in performance for various folds/datasets are not significant.

* **(4)** Random forest is a good model for this dataset because the RMSE is much lower than the mean of the target data.

---

## Problem 3

 The `experimental formation enthalpy` dataset consits of more than 100 features; some of them are definitely more important than others. In this problem, we explore the feature importance using `gradient boosting`.

### (a)

This feature importance we've discussed in class is called `impurity based feature importance`. There is another type of feature importance called `permutation feature importance`.
- Briefly describe what is `impurity based feature importance` and steps to compute it.
- Briefly describe what is `permutation feature importance` and steps to compute it.

### Answer

* `Impurity-based feature importance` is a technique used to evaluate the significance of each feature in a dataset to the model's prediction. It quantifies the influence of each feature in reducing error in the decision tree. Calculating steps:
    * Grow a decision tree
    * Calculate impurity of each leaf (Gini impurity for classification or error for regression)
    * Calculate total impurity = weighted average of impurities for all leaves

* `Permutation feature importance` is a technique used to evaluate the significant of each feature in a dataset to the model's prediction. It measures the impact of randomly shuffling the values of a certain feature while keeping other features unchanged.
    * Evaluate the initial performance of the model (unshuffled)
    * Select a feature to compute the importance score
    * Randomly shuffle the values of the selected feature across all data points in the dataset.
    * Evaluate the model after permutation by calculate the same metrics in the initial evaluation

### (b)

- Split the features dataset ( features `X` and target `y`) in problem 2 into a training set and a test set with a 8:2 ratio. You should get `X_train`, `X_test`, `y_train`,  and `y_test`.
- Fit a `gradient boosting` model to the `training set`, and evaluate the training and test `mean squared errors`. You can use `scklearn`'s `GradientBoostingRegressor`.
- What are the training and test `mean squared errors`? 
- How is the model performance compared to the random forest model in problem 2?

### Answer 

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [52]:
# define a gradient boosting fitting function for easy testing and training
def GradientBoostModeling(features, target, test_ratio = 0.2):
    X_train, X_test, y_train, y_test = train_test_split(features,target, test_size=test_ratio, random_state= 17)

    model = GradientBoostingRegressor()
    model.fit(X_train,y_train)

    # training set error
    y_pred = model.predict(X_train)
    mse = mean_squared_error(y_train,y_pred)
    print("MSE on the training set: ", mse)

    # test set error
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    print("MSE on the test set: ", mse)
    return model

In [53]:
gb_model = GradientBoostModeling(features_set, target_set)

MSE on the training set:  0.003766923157598716
MSE on the test set:  0.006919952438089583


$\Rightarrow$ The MSE is much lower than the RMSE from random forest, meaning the gradient boosting model performs better than random forest for this dataset

### (c)
- Obtain the feature importance from the `gradient boosting` model in (b). Which type of feature importance is it?
- What are the 10 most important features, and what are the 10 least important features?

### Answer

The feature performance is the impurity-based feature importance

In [54]:
feature_importance = gb_model.feature_importances_
sorted_idx = np.argsort(feature_importance)
feature_importance_name = np.array(features_names)[sorted_idx]

In [55]:
print("10 MOST important features: ")
for i in range(0,10):
    print("\t",feature_importance_name[i])

print("\n10 LEAST important features: ")
for i in range(0,10):
    print("\t",feature_importance_name[-1-i])


10 MOST important features: 
	 MagpieData minimum NfUnfilled
	 MagpieData mode GSmagmom
	 MagpieData minimum NpValence
	 MagpieData mode NsValence
	 MagpieData range NsValence
	 MagpieData maximum NsValence
	 MagpieData maximum Column
	 MagpieData range GSmagmom
	 MagpieData maximum GSmagmom
	 MagpieData minimum GSmagmom

10 LEAST important features: 
	 MagpieData avg_dev Electronegativity
	 MagpieData minimum GSvolume_pa
	 MagpieData maximum Electronegativity
	 MagpieData avg_dev NdValence
	 MagpieData avg_dev NpUnfilled
	 MagpieData range Column
	 MagpieData mode GSvolume_pa
	 MagpieData minimum Number
	 MagpieData minimum CovalentRadius
	 MagpieData mean NpUnfilled


### (d)

Remove the 100 least important features from the dataset, and repeat step (b).

- What are the training and test `mean squared errors`?
- Compare the results with those in (b). What happens to the test error? Any explanation?


### Answer

In [56]:
# delete 100 least important features
new_feature_set = np.delete(features_set,sorted_idx[-1:-101:-1],1)


In [57]:
# train and test the new model
new_gb_model = GradientBoostModeling(new_feature_set,target_set)

MSE on the training set:  0.01698344628502337
MSE on the test set:  0.024212839094280927


Compared to **(b)**, the MSE of the new dataset is considerably higher, indicating this dataset performs much worse than the initial dataset. Since 100 out of 132 features have been removed from the dataset, their weighted factors have been lost as well. Even though these are less important/impacting than the remaining features, in aggregate they still contribute significantly to the performance of the model.

### (e)  NO need to do this for HW2

- You might want to explore the feature importance using `permutation feature importance`, and compare the results with `impurity based feature importance`.
- You might want to compare the feature importance between `random forest` and `gradient boosting`.